In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import h5py
import pywt

In [3]:
# label_dict = {'good':0, 'holes_cuts':1, 'threaderror':2, 'oilstains_colorerror':3, 'wrinkles':4, 'foreignbodies':5}
file_path = r'/home/soucs/Python/textile-defect-inspection/dataset/textile_defect_data.hdF5'
images = h5py.File(file_path)['jute_defect_imgs'][:]
labels = h5py.File(file_path)['jute_defect_labels'][:]

def IMG(i):
    return resize_img(images[i])
def resize_img(img):
    dim = (500, 500)
    return cv2.resize(img,dim,interpolation=cv2.INTER_AREA)
def show(imgs):
    for i, img in enumerate(imgs):
        cv2.imshow('img'+str(i),img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def blur(img,k=5):
    ksize = (k,k)
    return cv2.blur(img,ksize)

#### SIFT

* Texture of image giving too many redundant keypoints
* Defect part not getting detected in most images
* Blurred image results in less no.of kp but not detecting defect at all

In [12]:
# Applying SIFT
img = blur(IMG(10))
sift = cv2.SIFT_create()
kp, des = sift.detectAndCompute(img, None)
sift_img = cv2.drawKeypoints(img, kp, 0, (0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
show([img,sift_img])

#### Co-occurence matrix

* Used in svm and giving 45% accuracy appx.

In [6]:
def coMat(img,npx = 32):
    converted_image = np.round(img / 255.0 * npx-1).astype(np.uint8)
    img = converted_image
    # Define the distance and angle offsets for co-occurrence
    d = 1
    theta = 0
    # Compute the co-occurrence matrix
    co_mat = np.zeros((npx, npx), dtype=np.uint32)
    for i in range(d, img.shape[0]):
        for j in range(d, img.shape[1]):
            i_index = img[i, j]
            j_index = img[i-d, j+theta]
            co_mat[i_index, j_index] += 1
    # Normalize and return the co-occurrence matrix
    co_mat = co_mat.astype(np.float64)
    co_mat /= np.sum(co_mat)
    return co_mat

def glcm(img,npx=32):
    converted_image = np.round(img / 255.0 * npx).astype(np.uint8)
    img = converted_image
    displacement = (1, 0)
    num_bins = npx
    co_matrix = cv2.calcHist([img, img], [0, 1], None, [num_bins, num_bins], [0, num_bins, 0, num_bins], accumulate=False)
    co_matrix_normalized = cv2.normalize(co_matrix, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    return co_matrix_normalized

# # Calculate contrast, energy, homogeneity, correlation from glcm
# def glcm(img,npx=32):
#     converted_image = np.round(img / 255.0 * npx).astype(np.uint8)
#     img = converted_image
#     displacement = (1, 0)
#     num_bins = npx
#     co_matrix = cv2.calcHist([img, img], [0, 1], None, [num_bins, num_bins], [0, num_bins, 0, num_bins], accumulate=False)
#     co_matrix_normalized = cv2.normalize(co_matrix, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
#     contrast = np.sum((co_matrix_normalized * np.arange(num_bins)[:, np.newaxis] - np.mean(co_matrix_normalized)) ** 2)
#     energy = np.sum(co_matrix_normalized ** 2)
#     homogeneity = np.sum(co_matrix_normalized / (1 + np.abs(np.arange(num_bins)[:, np.newaxis] - np.arange(num_bins))))
#     x, y = np.meshgrid(np.arange(num_bins), np.arange(num_bins))
#     correlation = np.sum((co_matrix_normalized * (x - np.mean(x)) * (y - np.mean(y))) / (np.std(x) * np.std(y)))    
#     return [contrast, energy, homogeneity, correlation]

img = IMG(10)
print(coMat(img).shape)
print(glcm(img).shape)

(32, 32)
(32, 32)


#### Laplacian, Canny Edge, Dilation/Erosion

* Try dilation to extract defect region
* Try Laplacian for same
* Try Canny edge for same

In [ ]:
# # Apply Canny edge detector
# edges = cv.Canny(img, 100, 200)
# kernel = np.ones((3, 3), np.uint8) # Create 3x3 kernel for dilation
# dilated = cv.dilate(edges, kernel, iterations=2) # Dilate edges using kernel

# Compute the Laplacian
# log = cv.Laplacian(img, cv.CV_64F)
# show([img,log])


# # Difference of texture tile from img to detect kernel
# # Define the kernel
# kernel = img[:50,:50]
# # kernel_img  = np.tile(kernel, (10, 10))
# # result = img-kernel_img

# max_li = set()
# for i in range(0,450,10):
#     for j in range(0,450,10):
#         kernel = img[i:50+i,j:50+j]
#         max_li.add(coMat(kernel))
#         print(i,j)
# print(max_li)

#### Wavelet

* Certain defects, especially wrinkles not getting detected properly

In [9]:
img = IMG(170)

# Perform 2D wavelet transform using Haar wavelet
coeffs = pywt.dwt2(img, 'haar')

# Split the coefficients into sub-bands
cA, (cH, cV, cD) = coeffs

# Display the resulting sub-bands
cv2.imshow('Img', img)
cv2.imshow('Approximation (low-pass)', cA)
cv2.imshow('Horizontal detail (high-pass)', cH)
cv2.imshow('Vertical detail (high-pass)', cV)
cv2.imshow('Diagonal detail (high-pass)', cD)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Gray Level Histogram Correlation

* Check for patterns in correlation b/w defect and defect-less regions' GLH

In [ ]:
# # Define the kernel
# kernel = img[:50,:50]

# # Calculate histogram
# hist0 = cv.calcHist([kernel],[0],None,[256],[0,256])
# hist1 = cv.calcHist([img[150:200,350:400]],[0],None,[256],[0,256])
# show([img])

# # Plot histogram
# plt.plot(hist0)
# plt.plot(hist1)
# plt.xlim([0, 256])
# plt.xlabel('Gray level')
# plt.ylabel('Frequency')
# plt.show()
# # show([img,kernel,result])

# # calculate correlation coefficient between the two distributions
# corr_coef = np.corrcoef(hist0.reshape(-1), hist1.reshape(-1))[0, 1]
# print(corr_coef)

#### Feature Selection

In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.feature_selection import SelectKBest, chi2

# label_dict = {'good':0, 'holes_cuts':1, 'threaderror':2, 'oilstains_colorerror':3, 'wrinkles':4, 'foreignbodies':5}
file_path = r'/home/soucs/Python/textile-defect-inspection/dataset/hist_features.csv'
df = pd.read_csv(file_path)

X = df.iloc[:,:-1]
y = df[['label']]

enc = OneHotEncoder()
y = enc.fit_transform(y).toarray()

data = np.concatenate([X.values,y],axis=1)

correlation_matrix = pd.DataFrame(data).corr()
print(correlation_matrix)

          0         1         2         3         4         5         6    
0   1.000000  0.019928  0.688875  0.688876  0.384084  0.088680 -0.125088  \
1   0.019928  1.000000  0.084938  0.084918  0.100814 -0.006650 -0.000049   
2   0.688875  0.084938  1.000000  1.000000  0.926884  0.090134  0.004733   
3   0.688876  0.084918  1.000000  1.000000  0.926884  0.090134  0.004733   
4   0.384084  0.100814  0.926884  0.926884  1.000000  0.062578  0.075802   
5   0.088680 -0.006650  0.090134  0.090134  0.062578  1.000000 -0.022459   
6  -0.125088 -0.000049  0.004733  0.004733  0.075802 -0.022459  1.000000   
7   0.088954 -0.047167  0.045371  0.045372 -0.000681  0.109110 -0.200200   
8   0.063308  0.047458  0.028102  0.028101 -0.011015 -0.020704 -0.199599   
9   0.033531 -0.035387  0.049471  0.049472  0.054070 -0.021407 -0.200200   
10  0.090074 -0.000049  0.028266  0.028266  0.003286 -0.022064 -0.200200   
11 -0.150653  0.035289 -0.155886 -0.155887 -0.121485 -0.022518 -0.200200   

          7